In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

# GPT는 계산을 못 한다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

# agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 우리의 tool이 한 개보다,더 많은 input을 가질 수 있다. 
# verbose=True agent가 하는 작업의 모든 과정 및 추론을 보여준다.
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION ,tools=[
    StructuredTool.from_function(func=plus, name="Sum Calculator", description="Use this to perform sums of two numbers"),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# Zero-shot React Agent

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

# handle_parsing_errors=True
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION , handle_parsing_errors=True, tools=[
    Tool.from_function(func=plus, name="Sum Calculator", description="Use this to perform sums of two numbers. Use this tool by sending a pair of numberts separated by a comma\
        \nExample:1,2"),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from typing import Type

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

class CalculatorToolArgsSchema(BaseModel):
    a: int = Field(description="The first number")
    b: int = Field(description="The second number")
    

class CalculatorTool(BaseTool):
    name = "SumCalculator"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema
    
    def _run(self, a, b):
        return plus(a, b)
    
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    CalculatorTool(),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# Search Tool

# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

from typing import Type

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for apple Company
    """
    
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    StockMarketSymbolSearchTool(),
])

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)

In [ ]:
# Search Tool

# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

from typing import Type

llm = ChatOpenAI(temperature=0.1, model_name = "gpt-3.5-turbo-1106")
alpha_vantage_api_key = os.environ.get("GI7UXI89QUWWJFK5")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

#

class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company.Example: AAPL,TSLA")

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    
class CompanyStockPerformanceTool(BaseTool):
    name="CompanyStockperformance"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    StockMarketSymbolSearchTool(),
    # 주식 심볼을 아는 데 사용
    CompanyOverviewTool(),
    # 정보를 얻는데 사용
    CompanyIncomeStatementTool(),
    # llm이 사용 안 할 수도 있다. llm이 더 많은 정보가 필요하다고 생각할 때만 사용
    CompanyStockPerformanceTool(),
])

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)

In [ ]:
from langchain.agents import create_sql_agent, agent_types
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(llm=llm, toolkit=toolkit, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke("Give me the 5 directors that have the highest grossing films.")

agent.invoke("Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.")